In [130]:
#All Libraries used
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

In [131]:
#Import Dataset
df = pd.read_csv('loan_default_prediction.csv')
print(df.shape)

(87500, 30)


In [132]:
#drop
vdrop=['ID','Validation','Designation','Debt_to_Income','Postal_Code','Deprecatory_Records',\
            'Inquiries','Gross_Collection','Sub_GGGrade','Total_Unpaid_CL','File_Status']
df=df.drop(vdrop,axis=1)
df.shape

(87500, 19)

In [133]:
def romanToInt(i):
    roman = {'I':1,'V':5,'X':10,'L':50,'C':100,'D':500,'M':1000,'IV':4,'IX':9,'XL':40,'XC':90,'CD':400,'CM':900}
    j = 0
    num = 0
    while j < len(i):
        if j+1<len(i) and i[j:j+2] in roman:
            num+=roman[i[j:j+2]]
            j+=2
        else:

            num+=roman[i[j]]
            j+=1
    return num

#Conversão dos anos de experiência para numérico
df['Experience']=df['Experience'].apply(lambda i: 10 if i[0:1]=='>' else 1 if i[0:1]=='<' else int(i[0:1]))
#Conversão da duração para numérico
df['Duration']=df['Duration'].apply(lambda i : i.replace(' years','')).astype(int)
#Conversão da GGGrade valor ordinal para numérico
df['GGGrade']=df['GGGrade'].apply(romanToInt).astype(int)
#ver resultado
#df.head()

In [134]:
#eliminar observações com pelo menos uma feature sem valores
df=df.dropna()
#drop duplicates
df.drop_duplicates()
df.shape

(77376, 19)

In [135]:
#df.describe()
df.head(5)

,Asst_Reg,GGGrade,Experience,Yearly_Income,Home_Status,Unpaid_2_years,Already_Defaulted,Lend_Amount,Interest_Charged,Usage_Rate,Present_Balance,State,Account_Open,Duration,Unpaid_Amount,Reason,Claim_Type,Due_Fee,Default
0,421802,2,10,633600.00,MORTGAGE,0,0,42023.25,15.39,88.924,607161.90,California,17,3,31216.05,debt consolidation,I,0.0,0
1,3964312,4,7,85483.20,RENT,0,0,38133.00,9.94,102.856,269234.06,NC,15,5,11660.49,debt consolidation,I,0.0,0
2,4247560,3,1,79200.00,RENT,0,0,17100.00,22.35,60.372,22476.53,Florida,7,5,5637.87,major purchase,I,0.0,0
3,197179,3,1,61600.00,RENT,0,0,5130.00,10.36,116.272,15242.09,NewJersey,9,3,15607.17,major purchase,I,0.0,1
4,4646684,5,2,68053.92,RENT,0,0,19665.00,13.68,127.280,65433.94,LA,10,5,27472.86,debt consolidation,I,0.0,0


In [136]:
vnumcont=['Asst_Reg','Experience','Yearly_Income','Lend_Amount','Interest_Charged','Usage_Rate','Present_Balance',\
            'Due_Fee','Unpaid_Amount']
vnumdisc=['Unpaid_2_years','Already_Defaulted','Account_Open','Duration']
vcatord=['Home_Status','State','Reason','Claim_Type','GGGrade']

In [140]:
#fazer histograma das categoricas e value_counts (verificar se há categorias de pouca relevancia)
for i in vcatord:
    print(df[i].value_counts())


MORTGAGE    36827
RENT        30219
OWN          7086
Name: Home_Status, dtype: int64
California    10512
TX             6134
Newyork        6087
Florida        4943
IL             2960
NewJersey      2701
PA             2693
Ohio           2544
GA             2465
NC             2139
VA             2091
MI             1934
Maryland       1752
AZ             1736
MA             1674
CO             1611
WA             1571
MN             1438
IN             1250
MO             1224
TN             1151
CT             1099
NV             1007
AL              976
WI              965
OR              899
LA              878
SC              855
KY              711
KS              700
OK              658
AR              555
UT              533
NM              413
HI              400
MS              362
NH              352
WV              340
RI              329
MT              221
DE              198
DC              196
AK              189
WY              167
SD              158
VT            

In [138]:
from scipy.stats import describe
describe(df[vnumcont], axis=0)

DescribeResult(nobs=77376, minmax=(array([2.4847e+04, 1.0000e+00, 8.8000e+03, 1.7100e+03, 8.0000e+00,
       0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00]), array([7.35184700e+06, 1.00000000e+01, 8.26403072e+06, 5.98500000e+04,
       2.50000000e+01, 1.53475600e+03, 5.62923433e+06, 1.69050000e+02,
       1.33877781e+06])), mean=array([3.80179301e+06, 6.14436001e+00, 1.34991476e+05, 2.59272131e+04,
       1.65183385e+01, 9.54898693e+01, 2.36378617e+05, 3.17506899e-01,
       2.95322809e+04]), variance=array([5.24006866e+12, 1.25192568e+01, 9.90654405e+09, 2.08621570e+08,
       2.40624090e+01, 1.64919577e+03, 6.52007679e+10, 1.26488043e+01,
       1.41194749e+09]), skewness=array([-8.89788695e-02, -1.93859572e-01,  1.72634447e+01,  6.39259733e-01,
       -4.11054226e-04,  4.07497980e-01,  2.47046383e+00,  1.67786120e+01,
        9.03167368e+00]), kurtosis=array([-1.41919775e+00, -1.58064266e+00,  9.54671663e+02, -3.19189156e-01,
       -1.19684817e+00,  1.96160885e+01,  1.43601803e+01

In [139]:
from scipy import stats
display(df.shape)
df=df[(df['Home_Status']!='OTHER')&(df['Home_Status']!='NONE')]
df=df[(np.abs(stats.zscore(df[vnumcont])) < 3).all(axis=1)]
df.shape


(77376, 19)

(74132, 19)

In [141]:
(df['Default']==1).sum()

14126

In [142]:
defaulted = df[df['Default']==1].sample(n=5000, random_state=101)
notdefault = df[df['Default']==0].sample(n=5000, random_state=101)
df = pd.concat([defaulted,notdefault],axis=0)
df.shape

(10000, 19)